Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 3

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

Instead, predict property sales prices for **One Family Dwellings** (`BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'`) using a subset of the data where the **sale price was more than \\$100 thousand and less than $2 million.** 

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do exploratory visualizations with Seaborn.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a linear regression model with multiple features.
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.


## Stretch Goals
- [ ] Add your own stretch goal(s) !
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way (without an excessive amount of formulas or academic pre-requisites).
(That book is good regardless of whether your cultural worldview is inferential statistics or predictive machine learning)
- [ ] Read Leo Breiman's paper, ["Statistical Modeling: The Two Cultures"](https://projecteuclid.org/download/pdf_1/euclid.ss/1009213726)
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html):

> Pipeline can be used to chain multiple estimators into one. This is useful as there is often a fixed sequence of steps in processing the data, for example feature selection, normalization and classification. Pipeline serves multiple purposes here:

> - **Convenience and encapsulation.** You only have to call fit and predict once on your data to fit a whole sequence of estimators.
> - **Joint parameter selection.** You can grid search over parameters of all estimators in the pipeline at once.
> - **Safety.** Pipelines help avoid leaking statistics from your test data into the trained model in cross-validation, by ensuring that the same samples are used to train the transformers and predictors.

In [2]:
# If you're in Colab...
import os, sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    # Install required python packages:
    # category_encoders, version >= 2.0
    # pandas-profiling, version >= 2.0
    # plotly, version >= 4.0
    !pip install --upgrade category_encoders pandas-profiling plotly
    
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Change into directory for module
    os.chdir('module3')

     |████████████████████████████████| 92kB 3.5MB/s 
     |████████████████████████████████| 133kB 8.6MB/s 
Requirement already up-to-date: plotly in /usr/local/lib/python3.6/dist-packages (4.1.1)
     |████████████████████████████████| 614kB 43.5MB/s 
     |████████████████████████████████| 225kB 42.3MB/s 
     |████████████████████████████████| 768kB 38.5MB/s 
     |████████████████████████████████| 194kB 46.7MB/s 
     |████████████████████████████████| 51kB 8.8MB/s 
     |████████████████████████████████| 737kB 34.7MB/s 
     |████████████████████████████████| 51kB 20.9MB/s 
  Created wheel for pandas-profiling: filename=pandas_profiling-2.3.0-py2.py3-none-any.whl size=145035 sha256=57ec0ac64a8eaca2f947c4046da60468f1c15679740b5f61685a5650b28d882d
  Stored in directory: /root/.cache/pip/wheels/ce/c7/f1/dbfef4848ebb048cb1d4a22d1ed0c62d8ff2523747235e19fe
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp36-none-any.whl size=27084 sha256=79ff893000440020836ec9b7ced58931918eef6db4

In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
#df = pd.read_csv('../data/NYC_Citywide_Rolling_Calendar_Sales.csv')
df = pd.read_csv('../data/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [30]:
print(df.shape)
df.head()

(23040, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,CHELSEA,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,nan,R4,"447 WEST 18TH STREET, PH12A",PH12A,"10,011",1,0,1,"10,733","1,979","2,007",2,R4,0,01/01/2019
1,1,FASHION,21 OFFICE BUILDINGS,4,812,68,nan,O5,144 WEST 37TH STREET,NaN,"10,018",0,6,6,"2,962","15,435","1,920",4,O5,0,01/01/2019
2,1,FASHION,21 OFFICE BUILDINGS,4,839,69,nan,O5,40 WEST 38TH STREET,NaN,"10,018",0,7,7,"2,074","11,332","1,930",4,O5,0,01/01/2019
3,1,GREENWICH VILLAGE-WEST,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,nan,R4,"1 SHERIDAN SQUARE, 8C",8C,"10,014",1,0,1,0,500,0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,nan,R1,"20 EAST 65TH STREET, B",B,"10,065",1,0,1,0,"6,406",0,2,R1,0,01/01/2019


In [31]:
print(df.shape)
df.describe(include= 'all')

(23040, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
count,"23,040",23040,23040,23039,"23,040","23,040",0,23039,23040,5201,"23,039","23,039","23,039","23,039",22987,"23,039","23,005","23,040",23040,"23,040",23040
unique,nan,252,44,10,nan,nan,nan,146,22691,1724,nan,nan,nan,nan,3652,nan,nan,nan,147,nan,120
top,nan,FLUSHING-NORTH,01 ONE FAMILY DWELLINGS,1,nan,nan,nan,D4,100 JEROME STREET,4,nan,nan,nan,nan,0,nan,nan,nan,D4,nan,01/24/2019
freq,nan,685,5061,11071,nan,nan,nan,3408,6,99,nan,nan,nan,nan,7500,nan,nan,nan,3408,nan,480
mean,3,NaN,NaN,NaN,"4,459",354,nan,NaN,NaN,NaN,"10,791",2,0,2,NaN,"3,509","1,820",2,NaN,"1,328,133",NaN
std,1,NaN,NaN,NaN,"3,713",628,nan,NaN,NaN,NaN,"1,103",10,6,12,NaN,"22,186",488,1,NaN,"10,253,944",NaN
min,1,NaN,NaN,NaN,1,1,nan,NaN,NaN,NaN,0,0,-148,0,NaN,0,0,1,NaN,0,NaN
25%,2,NaN,NaN,NaN,"1,341",22,nan,NaN,NaN,NaN,"10,306",0,0,1,NaN,494,"1,920",1,NaN,0,NaN
50%,3,NaN,NaN,NaN,"3,546",49,nan,NaN,NaN,NaN,"11,211",1,0,1,NaN,"1,356","1,940",1,NaN,"455,000",NaN
75%,4,NaN,NaN,NaN,"6,674",375,nan,NaN,NaN,NaN,"11,360",2,0,2,NaN,"2,280","1,965",2,NaN,"875,000",NaN


In [32]:
df['BUILDING_CLASS_CATEGORY'].value_counts()

01 ONE FAMILY DWELLINGS                       5061
02 TWO FAMILY DWELLINGS                       4567
10 COOPS - ELEVATOR APARTMENTS                3471
13 CONDOS - ELEVATOR APARTMENTS               3339
03 THREE FAMILY DWELLINGS                     1438
07 RENTALS - WALKUP APARTMENTS                 807
09 COOPS - WALKUP APARTMENTS                   672
15 CONDOS - 2-10 UNIT RESIDENTIAL              421
04 TAX CLASS 1 CONDOS                          418
44 CONDO PARKING                               366
17 CONDO COOPS                                 300
22 STORE BUILDINGS                             288
05 TAX CLASS 1 VACANT LAND                     288
12 CONDOS - WALKUP APARTMENTS                  256
14 RENTALS - 4-10 UNIT                         200
29 COMMERCIAL GARAGES                          147
08 RENTALS - ELEVATOR APARTMENTS               120
30 WAREHOUSES                                  105
21 OFFICE BUILDINGS                             96
31 COMMERCIAL VACANT LAND      

In [33]:
mask = df['BUILDING_CLASS_CATEGORY'].str.contains('ONE FAMILY DWELLINGS')
df=df[mask]
df.shape

(5061, 21)

In [34]:
df['SALE_PRICE'].describe()


count        5,061
mean       471,842
std        928,868
min              0
25%              0
50%        428,000
75%        640,000
max     30,139,360
Name: SALE_PRICE, dtype: float64

In [38]:
# sale price is hard to read in scientific notation
# change the formate with a comma seperator for thousand and zero decimal
pd.options.display.float_format = '{:,.0f}'.format

#create subset of the sale_price
df= df.query('SALE_PRICE >= 100000 &  SALE_PRICE <= 2000000')
df.describe()

,BOROUGH,BLOCK,LOT,EASE-MENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,SALE_PRICE
count,"3,164","3,164","3,164",0,"3,164","3,164","3,164","3,164","3,164","3,164","3,164","3,164"
mean,4,"6,909",76,nan,"11,027",1,0,1,"1,470","1,944",1,"628,190"
std,1,"3,964",161,nan,483,0,0,0,587,27,0,"296,913"
min,1,21,1,nan,"10,030",0,0,0,0,"1,890",1,"100,000"
25%,3,"4,003",21,nan,"10,461",1,0,1,"1,144","1,925",1,"445,000"
50%,4,"6,270",42,nan,"11,235",1,0,1,"1,360","1,938",1,"565,000"
75%,4,"10,206",69,nan,"11,413",1,0,1,"1,683","1,955",1,"760,000"
max,5,"16,350","2,720",nan,"11,697",2,2,3,"7,875","2,018",1,"2,000,000"


In [39]:
# 75% percentile GROSS_square _feet is 2575 but maximum is 7,875
#so look at the data with gross_square_feet >50000
df.query('GROSS_SQUARE_FEET > 5000')

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
4956,4,HOWARD BEACH,01 ONE FAMILY DWELLINGS,1,13984,18,nan,A1,158-28 86 STREET,NaN,"11,414",1,0,1,"5,000","5,184","2,003",1,A1,990000,2019-01-25
8414,2,RIVERDALE,01 ONE FAMILY DWELLINGS,1,5949,295,nan,A3,622 WEST 256TH STREET,NaN,"10,471",1,0,1,"12,235","7,500","2,018",1,A3,990000,2019-02-12
14491,5,TOTTENVILLE,01 ONE FAMILY DWELLINGS,1,8044,59,nan,A1,222 WOOD AVENUE,NaN,"10,307",1,0,1,"4,250","7,200","1,925",1,A1,667000,2019-03-13
14770,5,CLOVE LAKES,01 ONE FAMILY DWELLINGS,1,326,37,nan,A3,7 RICE AVENUE,NaN,"10,314",1,0,1,"17,500","7,875","1,950",1,A3,1850000,2019-03-14
15126,5,TODT HILL,01 ONE FAMILY DWELLINGS,1,894,186,nan,A3,77 COVENTRY ROAD,NaN,"10,304",1,0,1,"11,880","5,348","1,983",1,A3,1500000,2019-03-15
15388,5,GRYMES HILL,01 ONE FAMILY DWELLINGS,1,593,220,nan,A3,141 BERTHA PLACE,NaN,"10,301",1,0,1,"12,100","5,178","1,955",1,A3,575000,2019-03-18


### Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.

In [40]:
# train the model on training data because model learn to behave very well on train data but fail miserably on new sample
# to avoid overfiting split the data into train and test data,  create model on train data and test it on the test data
# let's check the SALE_DATE
# conver into datetime format and look at the date range
df['SALE_DATE'] = pd.to_datetime(df["SALE_DATE"], infer_datetime_format= True)
df['SALE_DATE'].describe()

count                    3164
unique                     91
top       2019-01-31 00:00:00
freq                       78
first     2019-01-01 00:00:00
last      2019-04-30 00:00:00
Name: SALE_DATE, dtype: object

In [41]:
df['SALE_DATE'].dt.month.value_counts()

1    952
3    802
2    763
4    647
Name: SALE_DATE, dtype: int64

In [42]:
cutoff = pd.to_datetime('2019-04-01')
train= df[df['SALE_DATE'] < cutoff]
test = df[df['SALE_DATE'] >= cutoff]
train.shape , test.shape

((2517, 21), (647, 21))

###Do exploratory visualizations with Seaborn.

In [0]:
import plotly.express as px
px.scatter(train, x= 'GROSS_SQUARE_FEET', y= 'SALE_PRICE', tredline=  color= 'SALE_PRICE')